In [10]:
#To remove all variables from the namespace
%reset -f

#Creating a log file to record the commands and outputs
%logstop
%logstart -t -o "E:/Python Clinical Course/Table4 log.txt"

Activating auto-logging. Current session state plus future input saved.
Filename       : E:/Python Clinical Course/Table4 log.txt
Mode           : backup
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [11]:
import pandas as pd
import pyreadstat
import os
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

adam_path = r"E:\Python Clinical Course\ADAM datasets\ADaM Datasets"

In [13]:
adam_datasets = {}

for file in os.listdir(adam_path):
    if file.endswith(".sas7bdat"):
        dataset_name = file.replace(".sas7bdat", "")
        file_path = os.path.join(adam_path, file)
        df, meta = pyreadstat.read_sas7bdat(file_path)
        adam_datasets[dataset_name] = df

In [14]:
# Load the data
adsl = adam_datasets.get("adsl")
advs = adam_datasets.get("advs")

In [15]:

advs = advs[(advs['SAFFL'] == 'Y') & advs['AVAL'].notna()]

# Step 2: Summarize Observed
obs = advs.groupby(['PARAMN', 'PARAM', 'TRT01AN', 'TRT01A', 'AVISITN', 'AVISIT'])['AVAL'].agg(
    n='count', mean='mean', median='median', std='std', min='min', max='max').reset_index()

# Format
for col in ['mean', 'median']:
    obs[col] = obs[col].map('{:.1f}'.format)
obs['std'] = obs['std'].map('{:.2f}'.format)
obs['min'] = obs['min'].map('{:.0f}'.format)
obs['max'] = obs['max'].map('{:.0f}'.format)

# Rename
obs = obs.rename(columns={
    'n': 'cn', 'mean': 'cmean', 'median': 'cmedian',
    'std': 'cstd', 'min': 'cmin', 'max': 'cmax'
})


In [16]:

# Step 3: Summarize Change from Baseline
chg = advs[(advs['AVISITN'] > 1) & advs['CHG'].notna()]
chg = chg.groupby(['PARAMN', 'PARAM', 'TRT01AN', 'TRT01A', 'AVISITN', 'AVISIT'])['CHG'].agg(
    n='count', mean='mean', median='median', std='std', min='min', max='max').reset_index()

# Format
for col in ['mean', 'median']:
    chg[col] = chg[col].map('{:.1f}'.format)
chg['std'] = chg['std'].map('{:.2f}'.format)
chg['min'] = chg['min'].map('{:.0f}'.format)
chg['max'] = chg['max'].map('{:.0f}'.format)

chg = chg.rename(columns={
    'n': 'chn', 'mean': 'chmean', 'median': 'chmedian',
    'std': 'chstd', 'min': 'chmin', 'max': 'chmax'
})

# Step 4: Merge observed and change summaries
final = pd.merge(obs, chg, on=['PARAMN', 'PARAM', 'TRT01AN', 'TRT01A', 'AVISITN', 'AVISIT'], how='left')


In [17]:

# Step 5: Pagination logic
final = final.sort_values(by=['PARAMN', 'AVISITN', 'TRT01AN'])
final = final.reset_index(drop=True)
final['lnt'] = final.index % 15 + 1
final['page1'] = final.index // 15 + 1


In [18]:

# ---------------- Word Export ----------------

doc = Document()

# Set margins
section = doc.sections[0]
section.top_margin = Inches(1)
section.bottom_margin = Inches(1)
section.left_margin = Inches(1)
section.right_margin = Inches(1)

# Set font
style = doc.styles['Normal']
style.font.name = 'Courier New'
style.font.size = Pt(9)
style.element.rPr.rFonts.set(qn('w:eastAsia'), 'Courier New')

# Titles
doc.add_paragraph("COVID-19 AA").alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
doc.add_paragraph("Protocol: 043").alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
doc.add_paragraph("Table 14.1.8 Summary of Changes in Vital Signs from Baseline to Final Visit (Safety Population)").alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Per-parameter pagination
for page in sorted(final['page1'].unique()):
    sub = final[final['page1'] == page]

    table = doc.add_table(rows=1, cols=14)
    table.style = 'Light Shading Accent 1'

    # Headers
    headers = [
        'Treatment', 'Visit',
        'n', 'Mean', 'Median', 'SD', 'Min', 'Max',  # Observed
        'n', 'Mean', 'Median', 'SD', 'Min', 'Max'   # Change
    ]
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Treatment'
    hdr_cells[1].text = 'Visit'
    hdr_cells[2].text = 'n'
    hdr_cells[3].text = 'Mean'
    hdr_cells[4].text = 'Median'
    hdr_cells[5].text = 'SD'
    hdr_cells[6].text = 'Min'
    hdr_cells[7].text = 'Max'
    hdr_cells[8].text = 'n'
    hdr_cells[9].text = 'Mean'
    hdr_cells[10].text = 'Median'
    hdr_cells[11].text = 'SD'
    hdr_cells[12].text = 'Min'
    hdr_cells[13].text = 'Max'

    # Rows
    for _, row in sub.iterrows():
        r = table.add_row().cells
        r[0].text = str(row['TRT01A'])
        r[1].text = str(row['AVISIT'])
        r[2].text = str(row['cn'])
        r[3].text = str(row['cmean'])
        r[4].text = str(row['cmedian'])
        r[5].text = str(row['cstd'])
        r[6].text = str(row['cmin'])
        r[7].text = str(row['cmax'])
        r[8].text = str(row['chn']) if pd.notna(row['chn']) else ''
        r[9].text = str(row['chmean']) if pd.notna(row['chmean']) else ''
        r[10].text = str(row['chmedian']) if pd.notna(row['chmedian']) else ''
        r[11].text = str(row['chstd']) if pd.notna(row['chstd']) else ''
        r[12].text = str(row['chmin']) if pd.notna(row['chmin']) else ''
        r[13].text = str(row['chmax']) if pd.notna(row['chmax']) else ''

    # Footer line and page break
    #doc.add_paragraph("_____________________________________________________________________")
    doc.add_page_break()

# Add footnote
doc.add_paragraph(r"E:\TAB7_1.SAS").alignment = WD_PARAGRAPH_ALIGNMENT.LEFT


In [19]:

# Save
doc.save(r"E:\Python Clinical Course\TLF\output\t_14_1_8.docx")
